Importing libraries.

In [ ]:
import pandas as pd
import json
import numpy as np

Importing and cleaning covid dataset:

In [ ]:
df = pd.read_csv('dpc-covid19-ita-regioni.csv')

# converting values in the "data" column in pandas datetime values
datetimes = pd.to_datetime(df["data"])
df["data"] = datetimes

#filtering rows on "data" column
#only keeping rows with "2020-05-24 17:00:00" value
#to only get death and cases up until the 24th of may of 2020
start_date = "2020-05-24 17:00:00"
end_date = "2020-05-24 17:00:00"
after_start_date = df["data"] >= start_date
before_end_date = df["data"] <= end_date
between_two_dates = after_start_date & before_end_date
filtered_dates = df.loc[between_two_dates]

df = filtered_dates

#removing unnecessary columns
df_tot_pos_morti25052020 = df[['codice_regione', 'denominazione_regione', 'totale_positivi', 'deceduti']]

#ordering rows based on region code
df_tot_pos_morti25052020.sort_values(by=['codice_regione'], inplace=True)
df_tot_pos_morti25052020=df_tot_pos_morti25052020.reset_index(drop=True)

#merging P.A. Bolzano and P.A. Trento
tot_pos_trentino = df_tot_pos_morti25052020['totale_positivi'].values[19] + df_tot_pos_morti25052020['totale_positivi'].values[20]
deceduti_trentino = df_tot_pos_morti25052020['deceduti'].values[19] + df_tot_pos_morti25052020['deceduti'].values[20]

trentino = pd.DataFrame({'codice_regione':4, 'denominazione_regione':'Trentino-Alto Adige/Südtirol', 'totale_positivi':tot_pos_trentino, 'deceduti':deceduti_trentino}, index=[3])
df_tot_pos_morti25052020 = pd.concat([df_tot_pos_morti25052020.iloc[:3], trentino, df_tot_pos_morti25052020.iloc[3:]]).reset_index(drop=True)
df_tot_pos_morti25052020 = df_tot_pos_morti25052020.drop([20, 21])

Cleaning dataset regarding population numbers:

In [ ]:
df_pop = pd.read_csv('DCIS_POPORESBIL1_12012022143315331.csv')

df_pop = df_pop.loc[df_pop['TIME'] == '2020']
df_pop.reset_index(drop=True)
df_pop = df_pop[['Territorio', 'Value']]
df_pop = df_pop.rename(columns={'Territorio': 'denominazione_regione', 'Value': 'popolazione'})
df_pop = df_pop.reset_index(drop=True)
df_pop['denominazione_regione'] = df_pop['denominazione_regione'].replace({'Valle d\'Aosta / Vallée d\'Aoste':'Valle d\'Aosta', 'Friuli-Venezia Giulia':'Friuli Venezia Giulia'})
 
#merging P.A. Bolzano and P.A. Trento
pop_trentino = df_pop['popolazione'].values[4] + df_pop['popolazione'].values[5]
trentino = pd.DataFrame({'denominazione_regione':'Trentino-Alto Adige/Südtirol', 'popolazione':pop_trentino}, index=[3])
df_pop = pd.concat([df_pop.iloc[:3], trentino, df_pop.iloc[3:]]).reset_index(drop=True)
df_pop = df_pop.drop([5, 6])

Merging of population dataset with covid dataset:

In [ ]:
df_finale = pd.merge(df_tot_pos_morti25052020, df_pop, on="denominazione_regione")

Renaming regions with Istat names:

In [ ]:
df_cod = pd.read_csv('Codici-statistici-e-denominazioni-al-01_01_2022.csv', sep=';',encoding='latin-1')
df_cod = df_cod[['Denominazione Regione']]
df_cod = df_cod.drop_duplicates()
df_cod = df_cod.reset_index(drop=True)
df_cod

df_finale['denominazione_regione'] = df_cod['Denominazione Regione']

Adding column regarding number of cases and death for every 100000 residents:

In [ ]:
df_finale['totale_positivi_ogni_100000'] = df_finale.apply(lambda row: round((row.totale_positivi / row.popolazione)*100000), axis=1)
df_finale['deceduti_ogni_100000'] = df_finale.apply(lambda row: round((row.deceduti / row.popolazione)*100000), axis=1)

Translation of columns in english:

In [ ]:
df_finale = df_finale.rename(columns={'codice_regione': 'region_istat_code', 'denominazione_regione': 'region_name', 'totale_positivi': 'tot_positives', 'deceduti': 'deaths', 'popolazione':'population', 'totale_positivi_ogni_100000':'tot_positives_every_100000', 'deceduti_ogni_100000': 'deaths_every_100000'})

Importing and cleaning temperature dataset.

In [ ]:
df_temp = pd.read_csv('Tavola_1-Tabella 1.csv', sep=';')
df_temp = df_temp[['region_istat_code', 'temperature_mean']]
df_temp = df_temp.groupby('region_istat_code')['temperature_mean'].mean().round(2).reset_index(name='temperature_mean')

Importing and cleaning hospital dataset:

In [ ]:
df_san = pd.read_csv('C_17_dataset_68_0_upFileUTF8CODREG.csv', sep=';')
df_san = df_san.groupby(['Codice Regione'], sort=False).size().reset_index(name='hospital_count')
df_san = df_san.rename(columns={'Codice Regione': 'region_istat_code'})

Importing and cleaning surface dataset:

In [ ]:
df_sup = pd.read_csv('DCCV_CARGEOMOR_ST_COM_27032022165808849.csv')
df_sup = df_sup.rename(columns={'Territorio': 'region_name', 'Value': 'surface_km2'})
df_sup = df_sup[['region_name', 'surface_km2']]
df_sup.at[1,'region_name']='Valle d\'Aosta/Vallée d\'Aoste'
df_sup.at[4,'region_name']='Trentino-Alto Adige/Südtirol'

Importing and cleaning surface dataset:

In [ ]:
df_eta = pd.read_csv('DCIS_INDDEMOG1_28032022142732546.csv')

df_eta = df_eta.rename(columns={'Territorio': 'region_name', 'Value': 'age_mean'})
df_eta = df_eta[['region_name', 'age_mean']]
df_eta.at[1,'region_name']='Valle d\'Aosta/Vallée d\'Aoste'
df_eta.at[4,'region_name']='Trentino-Alto Adige/Südtirol'

Importing and cleaning pm10 ds:

In [ ]:
df_pm10 = pd.read_csv('DataExtract.csv')
df_net_r = pd.read_csv('air_quality_net_italy_regions.csv', sep=';')

df_pm10_r = pd.merge(df_pm10, df_net_r, on="Air Quality Network")
pm10r_fil = df_pm10_r[['Air Pollution Level', 'Region Code']]
pm10r_fil = pm10r_fil.groupby('Region Code', as_index=False)['Air Pollution Level'].mean().round(2)
pm10r_fil = pm10r_fil.rename(columns={'Region Code': 'region_istat_code', 'Air Pollution Level': 'pm10_mean_level'})
pm10r_fil = pm10r_fil.reset_index(drop=True)

Merging everything in final dataset:

In [ ]:
df_finale = pd.merge(df_finale, pm10r_fil, on="region_istat_code")

In [ ]:
df_finale = pd.merge(df_finale, df_sup)
df_finale['population_density'] = df_finale.apply(lambda row: round((row.population / row.surface_km2)), axis=1)

In [ ]:
df_finale = pd.merge(df_finale, df_eta)

In [ ]:
df_finale = pd.merge(df_finale, df_san, how='left')
df_finale['hospital_count'].fillna(0, inplace = True)
df_finale['hospital_count'] = df_finale['hospital_count'].apply(np.int64)

In [ ]:
df_finale = pd.merge(df_finale, df_temp, on="region_istat_code")

Creating CSV file:

In [ ]:
df_finale_csv = df_finale.to_csv('covid-data-regions-first-quarantine.csv', encoding='utf-8', index=False)

Creating GEOJSON file:

In [ ]:
df_dict = df_finale.to_dict(orient='records')

with open('limits_IT_regions.geojson', 'r') as f1:
    json_rs = json.load(f1)

jreg = json_rs.copy()

for feat in jreg['features']:
    f_pr = feat['properties'].copy()
    for i in df_dict:
        if f_pr['reg_istat_code_num'] == i['region_istat_code']:
            feat['properties'].clear()
            feat['properties'].update(i)
            
with open('covid-data-regions-first-quarantine.geojson', 'w') as f:
    json.dump(jreg, f)